# Preparation

In [ ]:
import numpy as np

import pandas as pd
from pandas import DataFrame

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag

import re

import os
import random
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
torch.cuda.empty_cache()

import sys
sys.path = [
    '../input/readability-package',
] + sys.path
import readability
import spacy

from sklearn import model_selection

import transformers
import torch
import pytorch_lightning as pl
from transformers import BertModel, BertTokenizer, BertForSequenceClassification
from transformers import RobertaTokenizer, RobertaModel, RobertaForSequenceClassification
from torch.utils.data import DataLoader, Dataset

import random

from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import KFold

import lightgbm as lgb

from fastprogress.fastprogress import  progress_bar

In [ ]:
train_df = pd.read_csv("../input/commonlitreadabilityprize/train.csv")
#train_df=train_df.head(10)
test_df = pd.read_csv("../input/commonlitreadabilityprize/test.csv")

train_df['excerpt'] = train_df['excerpt'].apply(lambda e: e.replace('\n', ''))
test_df['excerpt'] = test_df['excerpt'].apply(lambda e: e.replace('\n', ''))

# Cleaning Texts Function

In [ ]:
train_df['excerpt_preprocessed'] =  train_df['excerpt']
train_df['excerpt_preprocessed'] = train_df['excerpt_preprocessed'].replace('\s+', ' ', regex=True)

test_df["excerpt_preprocessed"] = train_df['excerpt']
test_df['excerpt_preprocessed'] = test_df['excerpt_preprocessed'].replace('\s+', ' ', regex=True)

In [ ]:
pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_colwidth', None)
train_df['excerpt_preprocessed'].head(3)

# Fetch some features

In [ ]:
#source: https://www.kaggle.com/ravishah1/readability-feature-engineering-non-nn-baseline/data

def readability_measurements(passage: str):
    """
    This function uses the readability library for feature engineering.
    It includes textual statistics, readability scales and metric, and some pos stats
    """
    results = readability.getmeasures(passage, lang='en')
    
    chars_per_word = results['sentence info']['characters_per_word']
    syll_per_word = results['sentence info']['syll_per_word']
    words_per_sent = results['sentence info']['words_per_sentence']
    
    
    tobeverb = results['word usage']['tobeverb']
    auxverb = results['word usage']['auxverb']
    conjunction = results['word usage']['conjunction']
    pronoun = results['word usage']['pronoun']
    preposition = results['word usage']['preposition']
    nominalization = results['word usage']['nominalization']
    
    pronoun_b = results['sentence beginnings']['pronoun']
    interrogative = results['sentence beginnings']['interrogative']
    article = results['sentence beginnings']['article']
    subordination = results['sentence beginnings']['subordination']
    conjunction_b = results['sentence beginnings']['conjunction']
    preposition_b = results['sentence beginnings']['preposition']

    
    return [chars_per_word, syll_per_word, words_per_sent,
            tobeverb, auxverb, conjunction, pronoun, preposition, nominalization,
            pronoun_b, interrogative, article, subordination, conjunction_b, preposition_b]

In [ ]:
def spacy_features(df: pd.DataFrame):
    """
    This function generates features using spacy en_core_wb_lg
    I learned about this from these resources:
    https://www.kaggle.com/konradb/linear-baseline-with-cv
    https://www.kaggle.com/anaverageengineer/comlrp-baseline-for-complete-beginners
    """
    
    nlp = spacy.load('en_core_web_lg')
    with nlp.disable_pipes():
        vectors = np.array([nlp(text).vector for text in df.excerpt])
        
    return vectors

def get_spacy_col_names():
    names = list()
    for i in range(300):
        names.append(f"spacy_{i}")
        
    return names

In [ ]:
def pos_tag_features(passage: str):
    """
    This function counts the number of times different parts of speech occur in an excerpt
    """
    pos_tags = ["CC", "CD", "DT", "EX", "FW", "IN", "JJ", "JJR", "JJS", "LS", "MD", 
                "NN", "NNS", "NNP", "NNPS", "PDT", "POS", "PRP", "RB", "RBR", "RBS", "RP", "TO", "UH",
                "VB", "VBD", "VBG", "VBZ", "WDT", "WP", "WRB"]
    
    tags = pos_tag(word_tokenize(passage))
    tag_list= list()
    
    for tag in pos_tags:
        tag_list.append(len([i[0] for i in tags if i[1] == tag]))
    
    return tag_list

In [ ]:
def generate_other_features(passage: str):
    """
    This function is where I test miscellaneous features
    This is experimental
    """
    # punctuation count
    hyphens = passage.count("-")
    periods = passage.count(".")
    commas = passage.count(",")
    semis = passage.count(";")
    exclaims = passage.count("!")
    questions = passage.count("?")
    
    # Some other stats
    num_char = len(passage)
    num_words = len(passage.split(" "))
    unique_words = len(set(passage.split(" ") ))
    word_diversity = unique_words/num_words
    
    word_len = [len(w) for w in passage.split(" ")]
    longest_word = np.max(word_len)
    avg_len_word = np.mean(word_len)
    
    return [hyphens,periods, commas, semis, exclaims, questions,
            num_char, num_words, unique_words, word_diversity,
            longest_word, avg_len_word]

In [ ]:
##Just testing

just_testingdf=pd.DataFrame(train_df["excerpt_preprocessed"].head().apply(lambda p : generate_other_features(p)).tolist(),
                                columns=["hyphens","periods", "commas", "semis", "exclaims", "questions",
                                         "num_char", "num_words", "unique_words", "word_diversity",
                                         "longest_word", "avg_len_word"])
just_testingdf

In [ ]:
!pip install ../input/pyphen011whl/pyphen-0.11.0-py3-none-any.whl
!pip install ../input/textstatwhl/textstat-0.7.1-py3-none-any.whl

In [ ]:
from nltk.corpus import stopwords
import spacy
import timeit
import re
import textstat


nlp = spacy.load('en_core_web_sm')
punct=";|!|:|;|,|-|'"
stop=set(stopwords.words('english'))

def preprocess_dataframe(df):
    #Set a unique Numbering for each exerpt
    df=df.reset_index()  
    #Average excerpt length
    train_df['excerpt_length']=train_df['excerpt'].str.len()
    avg_excerpt_len=round(train_df['excerpt_length'].mean(),0) #Avg. excerpt length
    #Convert all text to lowecase
    df['excerpt_preprocess']=df['excerpt_preprocessed'].str.lower()         
    #FEATURE ENGINEERING: Get the legth of each excerpt
    df['excerpt_actual_length']=df['excerpt_preprocess'].str.len()
    #Remove common words from excerpt
    df['excerpt_preprocess']=df['excerpt_preprocess'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop]))
    #FEATURE ENGINEERING: Get the legth of the preprocessed excerpt
    df['excerpt_preprocessed_length']=df['excerpt_preprocess'].str.len()
    #FEATURE ENGINEERING: Percent frequent words
    df['excerpt_stopword_freq']=(df['excerpt_actual_length']-df['excerpt_preprocessed_length'])/df['excerpt_actual_length']
    #FEATURE ENGINEERING: Get count of punctuations in the excerpt
    df['excerpt_punct_count']=df['excerpt'].apply(lambda x: len(re.findall(punct, x)))
    #Convert excerpt into setences
    df['excerpt_sentence'] = df['excerpt_preprocess'].apply(lambda x: list(nlp(x).sents))
    #Convert each setence of the exerpt into a pandas row
    df=df.explode('excerpt_sentence')
    #Convert spacy object to string object
    df['excerpt_sentence']=df['excerpt_sentence'].apply(lambda x: x.text)    
    ##FEATURE ENGINEERING: Get sentence length
    df['sentence_length']=df['excerpt_sentence'].str.len()
    ##FEATURE ENGINEERING: Get word count
    df['totalwords'] = df['excerpt_sentence'].str.split().map(len)
    ##FEATURE ENGINEERING: Get normalized word count
    df['normalized_word_count'] = round(df['sentence_length']/df['totalwords'],2)
    ##FEATURE ENGINEERING: Get normalized stopword frequency
    df['normalized_stopword_freq']=round(df['excerpt_stopword_freq']*avg_excerpt_len,1)
    ##FEATURE ENGINEERING: Get average senetence length
    df['avg sent length']=df[['sentence_length', 'index']].groupby(['index']).agg(['median'])
    ##FEATURE ENGINEERING: Get average senetence length
    df=df[['index','excerpt','avg sent length','normalized_word_count','normalized_stopword_freq']].drop_duplicates(subset ='index').set_index('index')
    
    #Features from textstat
    df['flesch_reading_ease']=df['excerpt'].apply(lambda x: textstat.flesch_reading_ease(x))
    df['smog_index']=df['excerpt'].apply(lambda x: textstat.smog_index(x))
    df['flesch_kincaid_grade']=df['excerpt'].apply(lambda x: textstat.flesch_kincaid_grade(x))
    df['coleman_liau_index']=df['excerpt'].apply(lambda x: textstat.coleman_liau_index(x))
    df['automated_readability_index']=df['excerpt'].apply(lambda x: textstat.automated_readability_index(x))
    df['dale_chall_readability_score']=df['excerpt'].apply(lambda x: textstat.dale_chall_readability_score(x))
    df['difficult_words']=df['excerpt'].apply(lambda x: textstat.difficult_words(x))
    df['linsear_write_formula']=df['excerpt'].apply(lambda x: textstat.linsear_write_formula(x))
    df['gunning_fog']=df['excerpt'].apply(lambda x: textstat.gunning_fog(x))
    df['text_standard']=df['excerpt'].apply(lambda x: textstat.text_standard(x, float_output=True))
    df['fernandez_huerta']=df['excerpt'].apply(lambda x: textstat.fernandez_huerta(x))
    df['szigriszt_pazos']=df['excerpt'].apply(lambda x: textstat.szigriszt_pazos(x))
    df['gutierrez_polini']=df['excerpt'].apply(lambda x: textstat.gutierrez_polini(x))
    df['crawford']=df['excerpt'].apply(lambda x: textstat.crawford(x))
    return df.drop(columns=['excerpt'])

In [ ]:
pd.set_option('display.max_colwidth', None)
train_df['excerpt_preprocessed'].head(2)

In [ ]:
preprocess_dataframe(train_df.head(3))

In [ ]:
pd.merge(just_testingdf, preprocess_dataframe(train_df.head()), left_index=True, right_index=True)

In [ ]:
def create_folds(data: pd.DataFrame, num_splits: int):
    """ 
    This function creates a kfold cross validation system based on this reference: 
    https://www.kaggle.com/abhishek/step-1-create-folds
    """
    # we create a new column called kfold and fill it with -1
    data["kfold"] = -1
    
    # the next step is to randomize the rows of the data
    data = data.sample(frac=1, random_state=42).reset_index(drop=True)

    # calculate number of bins by Sturge's rule
    # I take the floor of the value, you can also
    # just round it
    num_bins = int(np.floor(1 + np.log2(len(data))))
    
    # bin targets
    data.loc[:, "bins"] = pd.cut(
        data["target"], bins=num_bins, labels=False
    )
    
    # initiate the kfold class from model_selection module
    kf = model_selection.StratifiedKFold(n_splits=num_splits)
    
    # fill the new kfold column
    # note that, instead of targets, we use bins!
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)):
        data.loc[v_, 'kfold'] = f
    
    # drop the bins column
    data = data.drop("bins", axis=1)

    # return dataframe with folds
    return data

In [ ]:
class CLRDataset:
    """
    This is my CommonLit Readability Dataset.
    By calling the get_df method on an object of this class,
    you will have a fully feature engineered dataframe
    """
    def __init__(self, df: pd.DataFrame, train: bool, n_folds=2):
        self.df = df
        self.excerpts = df["excerpt_preprocessed"]
        
        self._extract_features()
        
        if train:
            self.df = create_folds(self.df, n_folds)
        
    def _extract_features(self):
        scores_df = pd.DataFrame(self.df["excerpt_preprocessed"].apply(lambda p : readability_measurements(p)).tolist(), 
                                 columns=["chars_per_word", "syll_per_word", "words_per_sent",
                                          "tobeverb", "auxverb", "conjunction", "pronoun", "preposition", "nominalization",
                                          "pronoun_b", "interrogative", "article", "subordination", "conjunction_b", "preposition_b"])
        self.df = pd.merge(self.df, scores_df, left_index=True, right_index=True)
        
        spacy_df = pd.DataFrame(spacy_features(self.df), columns=get_spacy_col_names())
        self.df = pd.merge(self.df, spacy_df, left_index=True, right_index=True)
        
        pos_df = pd.DataFrame(self.df["excerpt_preprocessed"].apply(lambda p : pos_tag_features(p)).tolist(),
                              columns=["CC", "CD", "DT", "EX", "FW", "IN", "JJ", "JJR", "JJS", "LS", "MD", 
                                       "NN", "NNS", "NNP", "NNPS", "PDT", "POS", "PRP", "RB", "RBR", "RBS", "RP", "TO", "UH",
                                       "VB", "VBD", "VBG", "VBZ", "WDT", "WP", "WRB"])
        self.df = pd.merge(self.df, pos_df, left_index=True, right_index=True)
        
        other_df = pd.DataFrame(self.df["excerpt_preprocessed"].apply(lambda p : generate_other_features(p)).tolist(),
                                columns=["hyphens","periods", "commas", "semis", "exclaims", "questions",
                                         "num_char", "num_words", "unique_words", "word_diversity",
                                         "longest_word", "avg_len_word"])
        
        self.df = pd.merge(self.df, other_df, left_index=True, right_index=True)
        
        self.df = pd.merge(self.df, preprocess_dataframe(self.df), left_index=True, right_index=True)
        
    def get_df(self):
        return self.df
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx: int):
        pass

In [ ]:
dataset = CLRDataset(train_df, train=True)
train_df = dataset.get_df()

train_df.head(3)

In [ ]:
test_dataset = CLRDataset(test_df, train=False)
test_df = test_dataset.get_df()

test_df.head()

# Vectorize By BERT Function

## Fine Tuning

In [ ]:
MODEL_PATH = '../input/huggingface-roberta/roberta-large'

tokenizer = RobertaTokenizer.from_pretrained(MODEL_PATH)

In [ ]:
class RobertaForSequenceClassification_pl(pl.LightningModule):
    def __init__(self, model_name, num_labels, lr):
        super().__init__()
        
        self.save_hyperparameters()
        
        self.roberta_sc = RobertaForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels
        )
        
    def training_step(self, batch, batch_idx):
        output = self.roberta_sc(**batch)
        loss = output.loss
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        output = self.roberta_sc(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss)
        
    def test_step(self, batch, batch_idx):
        labels = batch.pop('labels')
        output = self.roberta_sc(**batch)
        labels_predicted = output.logits.argmax(-1)
        num_correct = (labels_predicted == labels).sum().item()
        accuracy = num_correct / labels.size(0)
        self.log('accuracy', accuracy)
        
    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.hparams.lr)

In [ ]:
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model/'
)

trainer = pl.Trainer(
    gpus=1,
    max_epochs=25,
    callbacks=[checkpoint]
)

In [ ]:
def createRobertaFineDataSet(excerpts, targets):
    data = []    
    for excerpt, target in zip(excerpts, targets):
        encoding = tokenizer(
            excerpt,
            max_length = 240,
            padding='max_length',
            truncation=True
        )

        encoding['labels'] = target
        encoding = { k: torch.tensor(v) for k, v in encoding.items() }

        data.append(encoding)

    return data

In [ ]:
# update kfold values for fine tune
kfolds = []

for i in progress_bar(train_df.index):
    kfolds.append(i % 5)
    
train_df['kfold'] = kfolds

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model = RobertaForSequenceClassification_pl(
    MODEL_PATH,
    num_labels=1,
    lr=1e-5
)

In [ ]:
for i in progress_bar(train_df['kfold'].unique()):
    train_df_for_fine_tune = train_df[train_df['kfold'] != i]
    test_df_for_fine_tune = train_df[train_df['kfold'] == i]
    
    dataset_train = createRobertaFineDataSet(
        train_df_for_fine_tune['excerpt'],
        train_df_for_fine_tune['target']
    )
    
    dataset_val = createRobertaFineDataSet(
        test_df_for_fine_tune['excerpt'],
        test_df_for_fine_tune['target']
    )

    train_dataloader = DataLoader(
        dataset_train,
        batch_size=8,
        shuffle=True
    )
    val_dataloader = DataLoader(
        dataset_val, 
        batch_size=128
    )

    trainer.fit(model, train_dataloader, val_dataloader)

In [ ]:
best_model_path = checkpoint.best_model_path

model = RobertaForSequenceClassification_pl.load_from_checkpoint(
    best_model_path
)

FINE_TUNED_MODEL_PATH = '/kaggle/working/model_transformers'

model.roberta_sc.save_pretrained(FINE_TUNED_MODEL_PATH)

## Roberta interface

In [ ]:
class RobertaDataset(nn.Module):
    def __init__(self, df, tokenizer, max_len=128):
        self.excerpt = df['excerpt'].to_numpy()
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(
            self.excerpt[idx],
            return_tensors='pt',
            max_length=self.max_len,
            padding='max_length',
            truncation=True
        )
        return encode
    
    def __len__(self):
        return len(self.excerpt)
    

def get_embeddings(df, path, plot_losses=True, verbose=True):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"{device} is used")
            
    MODEL_PATH = path
    model = RobertaModel.from_pretrained(MODEL_PATH, num_labels=1)
    model.to(device)
    model.eval()
    
    ds = RobertaDataset(df, tokenizer, config['max_len'])
    dl = DataLoader(
        ds,
        batch_size=config["batch_size"],
        shuffle=False,
        num_workers = 4,
        pin_memory=True,
        drop_last=False
    )

    embeddings = list()
    with torch.no_grad():
        for i, inputs in progress_bar(list(enumerate(dl))):
            inputs = {key:val.reshape(val.shape[0], -1).to(device) for key, val in inputs.items()}
            outputs = model(**inputs)
            #outputs = outputs[0][:, -1].detach().cpu().numpy()
            outputs = np.sum(outputs[0][:, -4:].detach().cpu().numpy(),axis=1)
            embeddings.extend(outputs)
            
    return np.array(embeddings)

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONASSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
config = {
    'batch_size': 8,
    'max_len': 240,
    'seed': 42,
}
seed_everything(seed=config['seed'])

train_embeddings =  get_embeddings(train_df, FINE_TUNED_MODEL_PATH)
test_embeddings = get_embeddings(test_df, FINE_TUNED_MODEL_PATH)

In [ ]:
train_embeddings.shape

# Prepare train and test data

In [ ]:
pd.set_option('display.max_rows', 500)

best_feat_idx=train_df.filter(regex='^(?!.*spacy_).*$').corr().query('target < -0.2 | 0.2 < target')['target']
best_feat_idx

In [ ]:
columns=set(best_feat_idx.index.values)

intersection = columns. intersection(test_df.columns) #Find common elements of set and list.
columns = list(intersection)
columns

In [ ]:
X_train = pd.DataFrame(train_embeddings)
X_train = pd.concat([X_train, train_df[columns]], axis=1)

X_test = pd.DataFrame(test_embeddings)
X_test = pd.concat([X_test, test_df[columns]], axis=1)

In [ ]:
y_train = train_df[['target']]

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=71)

cv = list(kf.split(X_train, y_train))

# Light GBM

In [ ]:
params = {
    'boosting_type': 'gbdt',
    'metric': 'rmse',
    'objective': 'regression',
    'seed': 42,
    'learning_rate': 0.04,
    'max_depth': 6,
    'n_jobs': -1,
    'verbose': -1,
    'num_leaves': 10,
    'max_bin': 63,
    'feature_fraction': 0.25,
    'extra_trees': True,
    'path_smooth':0.1
}
pred = np.zeros(X_test.shape[0])
rmses = []

for tr_idx, val_idx in progress_bar(cv):
    x_tr, x_va = X_train.iloc[tr_idx], X_train.iloc[val_idx]
    y_tr, y_va = y_train.iloc[tr_idx], y_train.iloc[val_idx]

    train_set = lgb.Dataset(x_tr, y_tr)
    val_set = lgb.Dataset(x_va, y_va, reference=train_set)

    model = lgb.train(
        params,
        train_set, 
        num_boost_round=10000,
        early_stopping_rounds=50,
        valid_sets=[train_set, val_set], 
        verbose_eval=-1
    )

    y_pred = model.predict(x_va)
    rmse = np.sqrt(mse(y_va, y_pred))
    rmses.append(rmse)
    
    tmp_pred = model.predict(X_test)
    pred += tmp_pred / 5
    
print("\n", "Mean Fold RMSE:", np.mean(rmses))

In [ ]:
# check xgboost version
#import xgboost
#print(xgboost.__version__)

In [ ]:
# create an xgboost regression model
#model = xgboost.XGBRegressor(n_estimators=1000, max_depth=8, eta=0.1, subsample=0.7, colsample_bytree=0.8)

In [ ]:
# define model evaluation method
#cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
#scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

In [ ]:
# force scores to be positive
#scores = absolute(scores)
#print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

In [ ]:
predictions = pd.DataFrame()
predictions['id'] = test_df['id']
predictions['target'] = pred
predictions.to_csv("submission.csv", index=False)

predictions